# Import Library

In [21]:
import os
import pandas as pd
import time
import unicodedata
import argparse
import openpyxl import load_workbook

# Key Generator Library
from pbkdf2 import PBKDF2
from binascii import hexlify
from secrets import token_urlsafe

# Encrypt & Decrypt Library
from pyaes import AESModeOfOperationCTR
from binascii import unhexlify, hexlify

# OOP Function

In [38]:
class KeyGenerator:
    def __init__(self, enc_dec_file, password_stored):
        self.enc_dec_file = enc_dec_file
        self.password_stored = password_stored
        
    def key_generator(self):
        password = token_urlsafe(32)
        key = PBKDF2(password, self.password_stored).read(32)
        with open(self.enc_dec_file, 'w') as f:
            f.write(hexlify(key).decode('UTF-8'))

class EncryptDecrypt:
    def __init__(self, enc_dec_file, filename_input, sheet_name, column_name, filename_output):
        self.enc_dec_file = enc_dec_file
        self.filename_input = filename_input
        self.sheet_name = sheet_name
        self.column_name = column_name
        self.filename_output = filename_output
        
    def encrypt_text(self, plaintext, enc_dec_file):
        key = unhexlify(open(enc_dec_file, 'r').read())
        plaintext = unicodedata.normalize('NFKD', plaintext).encode('ascii', 'ignore')
        aes = AESModeOfOperationCTR(key)
        ciphertext = aes.encrypt(plaintext)
        encrypted = hexlify(ciphertext).decode()
        return encrypted

    def decrypt_text(self, ciphertext, enc_dec_file):
        key = unhexlify(open(enc_dec_file, 'r').read())
        aes = AESModeOfOperationCTR(key)
        decrypted = aes.decrypt(unhexlify(ciphertext)).decode()
        return decrypted

    def encryption_process(self):
        # Read excel file
        df = pd.read_excel(self.filename_input, sheet_name=self.sheet_name)
        for column in self.column_name:
            if column in df.columns:
                df[column] = df[column].apply(lambda x: self.encrypt_text(str(x), self.enc_dec_file) if x else None)
            else:
                print("Column '{}' not found in the Excel file.".format(column))
        
        # Save encrypted data to the existing Excel file
        with pd.ExcelWriter(self.filename_input, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            df.to_excel(writer, sheet_name=self.sheet_name, index=False)
        print("Encryption process completed. Encrypted data saved to", self.filename_input)

    def decryption_process(self):
        # Read encrypted Excel file
        df = pd.read_excel(self.filename_input, sheet_name=self.sheet_name)
        for column in self.column_name:
            if column in df.columns:
                df[column] = df[column].apply(lambda x: self.decrypt_text(str(x), self.enc_dec_file) if x else None)
            else:
                print("Column '{}' not found in the Excel file.".format(column))
        
        # Save decrypted data to the existing sheet
        with pd.ExcelWriter(self.filename_input, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            df.to_excel(writer, sheet_name=self.sheet_name, index=False)
        print("Decryption process completed. Decrypted data saved to", self.filename_input)


# Parameter

In [39]:
# Example usage

# Generate encryption key file
enc_dec_file = 'key_irisk_sanf.txt'
password_stored = 'irisksanfganra'

# Generate key process
key_gen = KeyGenerator(enc_dec_file, password_stored)
key_gen.key_generator()
print("Encryption key generated and saved to", enc_dec_file)

# Specify input and output file paths
filename_input = "00 Dataplex Sample V1 20230428.xlsx"
sheet_name = "Movic Dashboard"
column_name = ['Business Term']

# Encrypt data
encryptor = EncryptDecrypt(enc_dec_file, filename_input, sheet_name, column_name)
encryptor.encryption_process()

# # Decrypt data (Used when you want to decrpyt the column)
# decryptor = EncryptDecrypt(enc_dec_file, filename_output_encrypt, sheet_name, column_name)
# decryptor.decryption_process()

Encryption key generated and saved to key_irisk_sanf.txt
Encryption process completed. Encrypted data saved to 00 Dataplex Sample V1 20230428.xlsx
